# 01_Book_Intro

**Content:**
1) Download a dataset
2) Download a trained model
3) Feed a new example to the model to check results

# Import libraries

In [8]:
from fastai.vision.all import *  #  to get fns and classes needed to create a wide variety of computer vision models

# A complete system for creating and training NN for recognizing cats vs dogs
## Download a dataset, download and luanch a pretrained model
What does the code do:
1) A dataset "the Oxford-IIIT Pet Dataset" (https://www.google.com/url?q=http%3A%2F%2Fwww.robots.ox.ac.uk%2F%7Evgg%2Fdata%2Fpets%2F) is downloaded from the fast.ai datasets collection to the GPU server, and then extracted. The dataset contains 7,349 images of cats and dogs.
2) A pretrained model (trained on 1.3 million images) is downloaded from the internet.  
(These two steps only need to be run once on GPU server. If you run the cell again, it will use the dataset and model that have already been downloaded, rather than downloading them again)
3) The pretrained model is fine-tuned using the latest advances in transfer learning, to create a model that is specially customized for recognizing dogs and cats.

In [26]:
# download and extract the dataset
path = untar_data(URLs.PETS)/'images'

# fn for labeling cats
def is_cat(x): return x[0].isupper()

# define the kind and structure of the dataset
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

# create CNN (specify architecture, data, and metrics)
learn = vision_learner(dls, resnet34, metrics=error_rate)

# perform fine-tuning
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.167203,0.019018,0.007442,02:32


epoch,train_loss,valid_loss,error_rate,time
0,0.060676,0.037679,0.009472,03:48


_______________
Comment:
- path = returns a Path object with the extracted location of the dataset
- is_cat(x) = a fn which labels cats based on a filename rule provided by the dataset creators. In the Pet dataset, each image is labeled using its filename (e.g., "great_pyrenees_173.jpg"): it starts with an uppercase letter if the image is a cat, and a lowercase letter otherwise. is_cat returns x[0].isupper(), which evaluates to True if the first letter is uppercase (i.e., it's a cat).

dls
- dls = data loaders. tells fastai what kind of dataset we have and how it is structured.
- ImageDataLoaders = one of the different classes for different kinds of deep learning datasets and problems. The first part of the class name is (and generally) the type of data (e.g., image or text)
- from_name_func() =  tells fastai how to get labels from the filenames (which means that labels can be extracted using a function applied to the filename), and passing is_cat.
- path = where to find the data
- valid_pct = tells fastai to hold out 20% of the data (validation set), another 80% is training set. By default, the 20% that is held out is selected randomly.
- seed = sets the random seed to the same value every time we run this code, which means we get the same validation set every time we run it—this way, if we change our model and retrain it, we know that any differences are due to the changes to the model, not due to having a different random validation set.
- label_func=is_cat = tells fastai how to get the labels from the dataset -  to use the is_cat function we just defined.
- item_tfms = defines the Transforms that we need. A Transform contains code that is applied automatically during training; fastai includes many predefined Transforms. There are two kinds:  
  item_tfms are applied to each item (in this case, each item is resized to a 224-pixel square)  
  batch_tfms are applied to a batch of items at a time using the GPU, so they're particularly fast  
  Why 224 pixels? This is the standard size for historical reasons (old pretrained models require this size exactly), but you can pass pretty much anything. If you increase the size, you'll often get a model with better results (since it will be able to focus on more details), but at the price of speed and memory consumption.

learn
- dls = dataloaders' object, our data
- resnet34 = the model, the one of the fastai's build-in models. there are some standard architectures that work most of the time, and in this case we're using one called ResNet. The 34 in resnet34 refers to the number of layers in this variant of the architecture (other options are 18, 50, 101, and 152).
- metrics = a metric is a fn that measures the quality of the model's predictions using the validation set. In this case, we're using error_rate, which is a fn provided by fastai that tells you what percentage of images in the validation set are being classified incorrectly. Another common metric for classification is accuracy (which is just 1.0 - error_rate).
- vision_learner() = also has a parameter pretrained, which defaults to True. When using a pretrained model, vision_learner will remove the last layer, since that is always specifically customized to the original training task (i.e. ImageNet dataset classification), and replace it with one or more new layers with randomized weights, of an appropriate size for the dataset you are working with. This last part of the model is known as the head.

learn.fine_tune
- fine_tune() = since we've started with a pretrained model, we use fine_tune instead of the fit() because we don't want to throw away all those capabilities that were already trained.
_______________

## Check the results of the pretrained model
Let's check that this model actually works.
1) Upload an image of a dog or a cat (make sure that it is a clear photo of a single dog or a cat, and not a line drawing, cartoon, or similar):

In [29]:
# upload through the upload button:
import ipywidgets as widgets
uploader = widgets.FileUpload()
uploader

FileUpload(value=(), description='Upload')

In [31]:
# or if we can't click an upload button, we can fake it:
from types import SimpleNamespace
uploader = SimpleNamespace(data = ['/Users/hela/Downloads/cat.jpeg']) 

2) Pass the uploaded file to the model. NN will tell you whether it thinks it is a dog or a cat, and how confident it is:

In [32]:
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

Is this a cat?: True.
Probability it's a cat: 1.000000
